In [ ]:
!pip install git+https://github.com/datalogai/recurrentshop.git

In [ ]:
!pip install git+https://github.com/farizrahman4u/seq2seq.git

In [ ]:
!pip install keras==2.2.4

In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
import seq2seq
from seq2seq.models import SimpleSeq2Seq,Seq2Seq,AttentionSeq2Seq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ML2021

In [ ]:
import pandas as pd
import numpy as np
#import missingno as msno
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras import layers
from keras.models import Sequential, Model
from keras import optimizers
import random
from scipy.ndimage import gaussian_filter1d

In [ ]:
data=pd.read_csv('wtbdata_245days.csv')
data

In [ ]:
data=data[data['Day']>=36].reset_index()

In [ ]:
columns=['Wspd','Wdir','Etmp','Itmp','Ndir','Pab1','Pab2','Pab3','Prtv','Patv']
for column in columns:
  data[column].fillna(method='bfill', inplace=True)

todelete=[]
todelete.extend(list(data[(data['Wdir']>180) | (data['Wdir']<-180)].index))
todelete.extend(list(data[(data['Ndir']>720) | (data['Ndir']<-720)].index))
todelete.extend(list(data[(data['Patv']<0) & (data['Wspd']>2.5)].index))
todelete.extend(list(data[(data['Pab1']>89) | (data['Pab2']>89) | (data['Pab3']>89)].index))
todelete.extend(list(data[(data['Patv']<0)].index))

data['Patv'].loc[pd.unique(todelete)]=0
ana=data['Patv']

In [ ]:
time=[0 for i in range(144*210)]
for i in range(134):
  time=np.sum([time,ana[i*144*210:(i+1)*144*210].tolist()],axis=0).tolist()

In [ ]:
data=data[data['TurbID']==1]
data=data.drop(['index','TurbID','Day','Tmstamp'],axis=1)
data

In [ ]:
single_move_avg=gaussian_filter1d(data['Patv'],5)
farm_move_avg=gaussian_filter1d(time,5)

In [ ]:
data['Patv']=single_move_avg

In [ ]:
y_scaler = MinMaxScaler()
label = y_scaler.fit_transform(np.array(farm_move_avg).reshape(-1,1))

In [ ]:
x_scaler = MinMaxScaler()
data=x_scaler.fit_transform(data.values)

In [ ]:
def generator_val(num_turbine,data,label,lookback,delay,min_index,max_index,step,batch_size,test=False):

  i=min_index+lookback

  if test:
    rows=np.arange(i,i+batch_size,6)
  else:
    rows=np.arange(i,i+batch_size)

  samples=np.zeros((len(rows),lookback//step,len(data[0])))
  targets=np.zeros((len(rows),6))

  for j,row in enumerate(rows):
    start=rows[j]-lookback
    end=rows[j]

    power_sum=label[end+delay:end+delay+6].reshape(6,)

    indices=range(start,end,step)
    samples[j]=data[indices]
 
    targets[j]=power_sum

  return samples,targets

In [ ]:
lookback=1734
step=6
delay=282
num_turbine=1
time_slice=lookback//step

train_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=0,
           max_index=29664,
           step=step,
           batch_size=27931)

val_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=25920,
           max_index=27935,
           step=step,
           batch_size=288)

test_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=27936,
           max_index=29952,
           step=step,
           batch_size=283,
           test=True)

In [ ]:
model=Sequential([
  layers.Conv1D(32,6,input_shape=(time_slice,10)),
  layers.BatchNormalization(),
  layers.Activation('relu'),
  layers.AveragePooling1D(pool_size=2),

  layers.Conv1D(32,6),
  layers.BatchNormalization(),
  layers.Activation('relu'),
  layers.AveragePooling1D(pool_size=2),

  AttentionSeq2Seq(output_dim=1, hidden_dim=128, output_length=6,input_shape=(68,32), depth=3)

])
model.summary()

In [ ]:
history = model.fit(train_gen[0],train_gen[1].reshape(27931,6,1),batch_size=256,epochs=10)

In [ ]:
def generator_val(num_turbine,data,label,lookback,delay,min_index,max_index,step,batch_size,shuffle=False):

  i=min_index+lookback

  rows=np.arange(i,i+batch_size)

  samples=np.zeros((len(rows),lookback//step,len(data[0])))
  targets=np.zeros((len(rows),288))

  for j,row in enumerate(rows):
    start=rows[j]-lookback
    end=rows[j]
    power_sum=label[end:end+288].reshape(288,)

    indices=range(start,end,step)
    samples[j]=data[indices]

    targets[j]=power_sum

  return samples,targets

In [ ]:
lookback=14*144
step=6
delay=0
num_turbine=1
time_slice=lookback//step

train_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=0,
           max_index=29951,
           step=step,
           batch_size=27936)

val_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=25920,
           max_index=27935,
           step=step,
           batch_size=288)

test_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=27936,
           max_index=29952,
           step=step,
           batch_size=1)

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss="mse", metrics=["mae",'mse'])
history = model.fit(train_gen[0],train_gen[1],batch_size=128,epochs=10,validation_split=0.2)

In [ ]:
mae=history.history['mae']
val_mae=history.history['val_mae']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)

plt.figure()
plt.plot(epochs,mae,'bo',label='Taining mae')
plt.plot(epochs,val_mae,'b',label='validation mae')
plt.title=('Training and validation mae')
plt.legend()

plt.figure()
plt.plot(epochs,loss,'bo',label='Taining loss')
plt.plot(epochs,val_loss,'b',label='validation loss')
plt.title=('Training and validation loss')
plt.legend()


plt.show()

In [ ]:
result=model.evaluate(test_gen[0],test_gen[1].reshape(-1,6,1))

In [ ]:
prediction=model.predict(test_gen[0])

In [ ]:
c=model.predict(train_gen[0],batch_size=128)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(144*10),t.flatten()[:288*5],'r',label='train_predict')
plt.plot(range(144*10),label[2016:2016+144*10],'b',label='true')
#plt.plot(range(1440,1440+288,1),label[-288:],'b')
#plt.plot(range(1440,1440+288,1),prediction,'g',label='test_pre')
plt.legend()

plt.figure(figsize=(15,5))
plt.plot(range(144*10),t.flatten()[-144*10:],'r',label='train_predict')
plt.plot(range(144*10),label[28512:29952],'b',label='true')
plt.plot(range(1440,1440+288,1),label[-288:],'b')
plt.plot(range(1440,1440+288,1),prediction.flatten(),'g',label='test_predict')
plt.legend()
plt.show()